In [71]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb


In [72]:
df = pd.read_excel("../../data/interim/modeling.xlsx")
df.head()

,match_id,big_age,program_type,match_length,avg_cadence_day,early_stage_score,late_stage_score,sentiment_change,sentiment_trend,busyness_level,big_age_match_start,little_age_match_start,same_gender,race_similarity
0,a1v2J0000027CWYQA2,25,Site,5.5,35.000000,0.7650,0.9905,0.2255,Stable,3,18,12,True,0.000000
1,a1v2J0000027CWfQAM,26,Site,8.5,46.200000,0.9987,0.9913,-0.0074,Stable,3,19,11,True,0.166667
2,a1v2J0000027CWiQAM,27,Site,6.9,35.333333,0.9612,0.9893,0.0281,Stable,3,19,12,True,0.000000
3,a1v2J0000027CWoQAM,25,Site,7.2,43.750000,0.9972,0.4696,-0.5276,Declined,3,18,8,True,0.000000
4,a1v2J0000027CWpQAM,27,Site,7.4,45.750000,0.9995,0.9825,-0.0170,Stable,3,20,11,True,0.166667


In [73]:
df.drop(['match_id', 'sentiment_trend', "program_type"], axis=True, inplace=True)

In [74]:
X = df.drop(columns=['match_length'])  # Features (all columns except 'target')
y = df['match_length']  # Target variable


In [75]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling (optional, but often helpful for XGBoost performance)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)


In [76]:

# Train an XGBoost model
model = lgb.LGBMRegressor(objective='regression', random_state=42)

# Fit the model to the training data
model.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001090 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1050
[LightGBM] [Info] Number of data points in the train set: 2611, number of used features: 10
[LightGBM] [Info] Start training from score 23.614477


LGBMRegressor(objective='regression', random_state=42)

In [77]:

# Make predictions on the test set
from numpy import sqrt


y_pred = model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = sqrt(mean_squared_error(y_test, y_pred))
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 7.259173104514579


In [78]:
#  Get feature importances
feature_importances = model.feature_importances_

# Create a DataFrame to show feature importances with their respective columns
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,  # Column names (features)
    'Importance': feature_importances  # Feature importances
})

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df


,Feature,Importance
1,avg_cadence_day,956
0,big_age,492
6,big_age_match_start,456
2,early_stage_score,297
3,late_stage_score,238
7,little_age_match_start,180
9,race_similarity,162
4,sentiment_change,137
5,busyness_level,65
8,same_gender,17
